<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [1]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [2]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-03-26 21:55:43.148277: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 21:55:43.758453: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758496: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-26 21:55:43.758502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import sys
sys.path.append("..")

### Reading Data Files:

In [3]:
from mtsa import calculate_aucroc

2024-03-29 00:44:02.815340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

from mtsa import files_train_test_split
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_train

array([1., 1., 1., 1., 1.])

### MFFCMix Model:

In [8]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [9]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.52

### Hitachi Model:

In [9]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('demux2array', Demux2Array()),
                ('array2melspec',
                 Array2MelSpec(frames=5, hop_length=512, n_fft=1024, n_mels=64,
                               power=2.0, sampling_rate=None)),
                ('final_model',
                 <mtsa.models.hitachi.AutoEncoderMixin object at 0x7f66601473a0>)])

In [8]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 1ms/step


0.24

### RANSynCoders

#### Data preprocessing

#### Normalization

In [11]:
N = 10  #5 * round((input_vector_transpose.shape[1] / 3) / 5)  # 10 for both bootstrap sample size and number of estimators
encoder_layers = 1  # number of hidden layers for each encoder
decoder_layers = 2  # number of hidden layers for each decoder
z = int((N / 2) - 1)  # size of latent space
activation = 'relu'
output_activation = 'sigmoid'
S = 5  # Number of frequency components to fit to input signals
delta = 0.05
batch_size = 180
freq_warmup = 5  # pre-training epochs
sin_warmup = 5  # synchronization pre-training
epochs = 10

In [5]:
from mtsa import RANSynCoders

model = RANSynCoders(mono=False)

In [6]:
model.fit(X_train, y_train)

2024-03-29 00:44:19.432877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-29 00:44:19.433145: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-29 00:44:19.436786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf


Start of frequency pre-train epoch 0
pre-reconstruction_loss: 0.002317442
Start of frequency pre-train epoch 1
pre-reconstruction_loss: 0.0024951494
Start of frequency pre-train epoch 2
pre-reconstruction_loss: 0.0025805288
Start of frequency pre-train epoch 3
pre-reconstruction_loss: 0.0023030946
Start of frequency pre-train epoch 4
found common oscillations at period(s) = [20.118980273362652, 20.236784140969164, 3.939188223524368, 68.90625, 13.405885214007784]

Start of sine representation pre-train epoch 0
sine_loss: 0.004315955
Start of sine representation pre-train epoch 1
sine_loss: 0.004300952
Start of sine representation pre-train epoch 2
sine_loss: 0.005113235
Start of sine representation pre-train epoch 3
sine_loss: 0.004370967
Start of sine representation pre-train epoch 4
sine_loss: 0.0042735194
Start of epoch 0
sine_loss: 0.0043790634 upper_bound_loss: 0.0007719623 lower_bound_loss: 0.0008315525
Start of epoch 1
sine_loss: 0.00467209 upper_bound_loss: 0.001061937 lower_bo

KeyboardInterrupt: 

In [14]:
sins, synched, upper, lower = model.predict(X_test)

TypeError: predict() missing 1 required positional argument: 't'

'/data/matheus-coelho/mtsa/examples/sample_data/machine_type_1/id_00/normal/00000113.wav'

In [7]:
auc = calculate_aucroc(model, X_test, y_test)
auc

[[[-5.1212031e-03 -4.7616027e-03 -3.0358613e-03 ...  1.3472688e-03
    1.3256151e-03  8.3333073e-04]
  [-3.7736979e-03 -3.8276827e-03 -3.6687795e-03 ...  1.9587947e-03
    1.7657288e-03  8.8460196e-04]
  [-3.5508475e-03 -3.6966032e-03 -2.9801929e-03 ...  2.6283236e-03
    2.8925634e-03  1.5666587e-03]
  ...
  [-3.2073138e-03 -4.1453699e-03 -3.4906270e-03 ...  4.0801563e-03
    5.2534607e-03  3.4237204e-03]
  [-4.0138112e-03 -4.7366568e-03 -4.0532481e-03 ...  2.9989968e-03
    3.7180518e-03  2.6368431e-03]
  [-5.4895412e-03 -5.3638159e-03 -3.5861621e-03 ...  1.2285613e-03
    2.0840410e-03  1.6545122e-03]]

 [[ 4.9227583e-03  5.7454007e-03  6.7333365e-03 ... -7.9784952e-03
   -8.9933593e-03 -5.8389502e-03]
  [ 2.3233306e-03  3.9213933e-03  5.0449492e-03 ... -7.6054209e-03
   -7.8123091e-03 -4.2766435e-03]
  [ 3.0960834e-03  2.3056280e-03  2.3028182e-03 ... -6.4535807e-03
   -4.9053729e-03 -2.5677814e-03]
  ...
  [ 5.2865800e-03  5.2039684e-03  4.2954171e-03 ... -7.4419146e-03
   -7.5509

InvalidParameterError: The 'y_score' parameter of roc_curve must be an array-like. Got None instead.